In [ ]:
%tensorflow_version 2.x

import os
import tensorflow
from tensorflow import keras

In [ ]:
import zipfile

drive_file = "/content/drive/MyDrive/Colab Data/MiniProject21/fnf_train.zip"
unzipper = zipfile.ZipFile(drive_file, "r")
unzipper.extractall("/tmp/train")
unzipper.close()

drive_file = "/content/drive/MyDrive/Colab Data/MiniProject21/fnf_test.zip"
unzipper = zipfile.ZipFile(drive_file, "r")
unzipper.extractall("/tmp/test")
unzipper.close()

In [ ]:
TRAIN_DIR = "/tmp/train/fnf_train"
VAL_DIR = "/tmp/test/fnf_test"

print("Train count:", len(os.listdir(TRAIN_DIR)))
print("Test count:", len(os.listdir(VAL_DIR)))

Train count: 2
Test count: 2


In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2021-04-16 04:51:25--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 108.177.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  35.3MB/s    in 2.4s    

2021-04-16 04:51:29 (35.3 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

weights_file = "/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"

# Define InceptionV3 object
inception = InceptionV3(input_shape=(224, 224, 3),  # Input image size
                        include_top=False,  # We will configure top layer manually
                        weights=None)  # we will used the downloaded weights file
inception.load_weights(weights_file)

# The model is used for feature extraction
# Hence we dont need to train the model again
for layer in inception.layers:
    layer.trainable = False


In [ ]:
# defining the last layer
last_layer = inception.get_layer("mixed7")
print("Last layer:", last_layer)

last_layer = last_layer.output

Last layer: <tensorflow.python.keras.layers.merge.Concatenate object at 0x7f73f2252250>


In [ ]:
# Flattening the last_layer into one dimension
x = layers.Flatten()(last_layer)

# Fully connected layer with 2048 nodes
# x = layers.Dense(2048, activation="relu")(x)

# Fully connected layer with 1024 nodes
x = layers.Dense(1024, activation="relu")(x)

# Dropout layer to prevent overfitting
# Dropout rate of 0.3 i..e 30% of nodes will be randomly deactivated
x = layers.Dropout(0.3)(x)

# Output layer
# Softmax activation: sum of probabilities of all classes will be 1
output_layer = layers.Dense(2, activation="softmax")(x)

In [ ]:
model = Model(inception.input, output_layer)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=["accuracy"])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Applying various filters on the image so that the model never sees the same 
# image twice and thus preventing overfitting
train_format = ImageDataGenerator(rescale=1/255,  # pixel value between 0 to 1
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

val_format = ImageDataGenerator(rescale=1/255)

train_data_generator = train_format.flow_from_directory(TRAIN_DIR,
                                                        target_size=(224, 224),
                                                        class_mode="categorical")

val_data_generator = val_format.flow_from_directory(VAL_DIR,
                                                    target_size=(224, 224),
                                                    class_mode="categorical")

Found 13108 images belonging to 2 classes.
Found 3277 images belonging to 2 classes.


In [ ]:
result = model.fit(train_data_generator,
                   epochs=10,
                   validation_data=val_data_generator,
                   verbose=2)

Epoch 1/10
410/410 - 2069s - loss: 0.2502 - accuracy: 0.9394 - val_loss: 0.1688 - val_accuracy: 0.9661
Epoch 2/10
410/410 - 2151s - loss: 0.1702 - accuracy: 0.9614 - val_loss: 0.1754 - val_accuracy: 0.9594
Epoch 3/10
410/410 - 2151s - loss: 0.1393 - accuracy: 0.9648 - val_loss: 0.1304 - val_accuracy: 0.9713
Epoch 4/10
410/410 - 2138s - loss: 0.1424 - accuracy: 0.9673 - val_loss: 0.1329 - val_accuracy: 0.9731
Epoch 5/10
410/410 - 2145s - loss: 0.1238 - accuracy: 0.9691 - val_loss: 0.1228 - val_accuracy: 0.9722
Epoch 6/10
410/410 - 2171s - loss: 0.1198 - accuracy: 0.9718 - val_loss: 0.1220 - val_accuracy: 0.9707
Epoch 7/10
410/410 - 2165s - loss: 0.1114 - accuracy: 0.9741 - val_loss: 0.1382 - val_accuracy: 0.9713
Epoch 8/10
410/410 - 2165s - loss: 0.1089 - accuracy: 0.9736 - val_loss: 0.1226 - val_accuracy: 0.9759
Epoch 9/10
410/410 - 2148s - loss: 0.1037 - accuracy: 0.9751 - val_loss: 0.1505 - val_accuracy: 0.9716
Epoch 10/10
410/410 - 2151s - loss: 0.1054 - accuracy: 0.9751 - val_loss:

Saving the model

In [ ]:
model.save("/content/drive/MyDrive/Colab Data/MiniProject21/food_non_food_1.h5")

Loading the saved model

In [ ]:
from tensorflow import keras

model = keras.models.load_model("/content/drive/MyDrive/Colab Data/MiniProject21/food_non_food_1.h5")

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np

image_path = "/content/drive/MyDrive/Colab Data/MiniProject21/test/006.jpg"
image = load_img(image_path, target_size=(224, 224))
x = img_to_array(image)
x = x.reshape((1, ) + x.shape)

x /= 255

# print(np.argmax(model.predict(x)[0]))
print(model.predict(x))
print(np.argmax(model.predict(x)[0]))

[[1.0000000e+00 2.0355935e-10]]
0
